# 法律助理
测试和实验GPT3.5在中国法律助理场景的应用：对中国法律的理解，对法律条款和陈述的综合分析能力的测试。
对于这个场景prompt设计的探索和测试


In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import LLMChain
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [2]:
llm = OpenAI(model_name='text-davinci-003', temperature=0.0, max_tokens=1500)
chat = ChatOpenAI(temperature=0)

最简单的推理测试：找到好的prompt很不容易，需要对法律场景、问题类型、模型与人在这样问题上的思维方式及其偏见（inductive bias）有深刻的理解后（经过很多尝试后）才能生成。

In [3]:
question = "以上法律条款适用于以上陈述的先决条件是什么？这个条件满足了吗？为什么？陈述是正确的吗？为什么？"

law="犯罪嫌疑人自被侦查机关第一次讯问或者采取强制措施之日起，有权委托辩护人; \
在侦查期间，只能委托律师作为辩护人。被告人有权随时委托辩护人."
prefix = "关于辩护人的权利与义务的表述"
statement ="犯罪嫌疑人高某因为绑架罪被立案侦查，\
自被第一次讯问后或被采取强制措施之日起，高某有权聘请其在医院工作的朋友袁某作其辩护人。"

In [4]:
qTemplate="""请仔细理解以下法律条款和陈述后回答问题。请用英文回答。\
法律条款、陈述和问题都被三重反引号分隔开\
法律条款：'''{law}''' \
陈述：'''{prefix} {statement}''' \
问题：'''{question}''' """

question = """以上法律条款适用于以上陈述的先决条件是什么？每个条件满足了吗？用bullet point列举出来？陈述是正确的吗？为什么？请用英文回答。expand to a one page analysis.""" 

qPrompt = PromptTemplate(input_variables=["law","prefix", "statement", "question"], template=qTemplate)

qChain = LLMChain(llm=chat, verbose=True, prompt=qPrompt)
qChain.run(law=law, prefix=prefix, statement=statement, question=question)



> Entering new LLMChain chain...
Prompt after formatting:
请仔细理解以下法律条款和陈述后回答问题。请用英文回答。法律条款、陈述和问题都被三重反引号分隔开法律条款：'''犯罪嫌疑人自被侦查机关第一次讯问或者采取强制措施之日起，有权委托辩护人; 在侦查期间，只能委托律师作为辩护人。被告人有权随时委托辩护人.''' 陈述：'''关于辩护人的权利与义务的表述 犯罪嫌疑人高某因为绑架罪被立案侦查，自被第一次讯问后或被采取强制措施之日起，高某有权聘请其在医院工作的朋友袁某作其辩护人。''' 问题：'''以上法律条款适用于以上陈述的先决条件是什么？每个条件满足了吗？用bullet point列举出来？陈述是正确的吗？为什么？请用英文回答。expand to a one page analysis.''' 

> Finished chain.


'The legal provisions stated in the first paragraph of the prompt outline the rights of criminal suspects and defendants to appoint a defense counsel. The key conditions that must be met for these provisions to apply are:\n\n- The individual must be a criminal suspect or defendant, meaning they are under investigation or have been formally charged with a crime.\n- The individual must have been questioned by law enforcement or subjected to coercive measures, such as arrest or detention.\n- If the individual is a suspect during the investigation phase, they can only appoint a lawyer as their defense counsel.\n\nIn the given statement, the individual named "高某" is a criminal suspect who has been investigated for kidnapping. The statement also indicates that they have been questioned or subjected to coercive measures, which satisfies the second condition. However, it is not clear from the statement whether the investigation is still in progress or if formal charges have been filed. If the 

In [5]:
qTemplate="""请把以下的多项选择题改为多道正确与错误的判断题.用bullet point列出所有的题但是不做解答。\
问题：'''{problem}''' """

problem = """关于暂予监外执行的说法，下列选项正确的是?  ：\
A.张某因盗窃被判处5年有期徒刑，在交付执 行前，突患严重疾病，需保外就医，应当由省级以上 监狱管理机关决定张某暂予监外执行\
B.不符合暂予监外执行条件的罪犯通过贿赂等非法手段被暂予监外执行的，在监外执行的期间不计人执行刑期\
C.罪犯在暂予监外执行期间脱逃的，脱逃的期间不计人执行刑期。被决定收监执行的社区矫正人员在逃的，由社区矫正机构负责追捕\
D.对暂予监外执行的罪犯，由公安机关交所在单位或者基层组织予以考察""" 

qPrompt = PromptTemplate(input_variables=["problem"], template=qTemplate)

qChain = LLMChain(llm=chat, verbose=True, prompt=qPrompt)
qChain.run(problem=problem)



> Entering new LLMChain chain...
Prompt after formatting:
请把以下的多项选择题改为多道正确与错误的判断题.用bullet point列出所有的题但是不做解答。问题：'''关于暂予监外执行的说法，下列选项正确的是?  ：A.张某因盗窃被判处5年有期徒刑，在交付执 行前，突患严重疾病，需保外就医，应当由省级以上 监狱管理机关决定张某暂予监外执行B.不符合暂予监外执行条件的罪犯通过贿赂等非法手段被暂予监外执行的，在监外执行的期间不计人执行刑期C.罪犯在暂予监外执行期间脱逃的，脱逃的期间不计人执行刑期。被决定收监执行的社区矫正人员在逃的，由社区矫正机构负责追捕D.对暂予监外执行的罪犯，由公安机关交所在单位或者基层组织予以考察''' 

> Finished chain.


'- 张某因盗窃被判处5年有期徒刑，在交付执行前，突患严重疾病，需保外就医，应当由省级以上监狱管理机关决定张某暂予监外执行（正确）\n- 不符合暂予监外执行条件的罪犯通过贿赂等非法手段被暂予监外执行的，在监外执行的期间不计人执行刑期（错误）\n- 罪犯在暂予监外执行期间脱逃的，脱逃的期间不计人执行刑期。被决定收监执行的社区矫正人员在逃的，由社区矫正机构负责追捕（正确）\n- 对暂予监外执行的罪犯，由公安机关交所在单位或者基层组织予以考察（正确）'